In [4]:
from market_analysis.models import Candle, UserProfile, Symbol, SortedStocksList, StrategyTimestamp
# from market_analysis import views
from django.contrib.auth.models import User
from heystox_intraday.select_stocks_for_trading import *
import pandas as pd
# from heystox_trade.settings import *
from datetime import datetime, timedelta
from heystox_intraday.intraday_fetchdata import *
from market_analysis.tasks.stock_data_import_tasks import *
from market_analysis.tasks.day_trading_tasks import *
from django.core.cache import cache, caches
from datetime import time, timedelta, datetime
from heystox_intraday.trading_indicator import *
import time

In [5]:
sorted_stocks = SortedStocksList.objects.filter(created_at__date=datetime.now().date())
sorted_stock = SortedStocksList.objects.get(symbol__symbol="tatamotors", created_at__date=datetime.now().date())
print(sorted_stock.created_at)
print(sorted_stock.entry_type)

2020-02-05 09:52:04.305890
BUY


In [6]:
# MACD Crossover
today_date = datetime.today().date()
df = get_macd_data(sorted_stock.symbol)
df.loc[(df["signal"] != df["signal"].shift()) & (df["signal"] == "BUY"), "signal"] = "BUY_CROSSOVER"
df.loc[(df["signal"] != df["signal"].shift()) & (df["signal"] == "SELL"), "signal"] = "SELL_CROSSOVER"
df = df.loc[df["date"] > str(today_date)]
new_df = df.copy(deep=True).drop(df.tail(1).index)
try:
    if sorted_stock.entry_type == "SELL":
        last_crossover = new_df[new_df.signal.str.endswith("SELL_CROSSOVER")].iloc[-1]
    elif sorted_stock.entry_type == "BUY":
        last_crossover = new_df[new_df.signal.str.endswith("BUY_CROSSOVER")].iloc[-1]
except:
    last_crossover = None
if last_crossover is not None:
    df_after_last_crossover = df.loc[df["date"] > last_crossover.date]
    try:
        if last_crossover.signal == "SELL_CROSSOVER":
            crossover_signal = df_after_last_crossover.loc[(df.macd_diff <= -0.070)].iloc[0]
        elif last_crossover.signal == "BUY_CROSSOVER":
            crossover_signal = df_after_last_crossover.loc[(df.macd_diff >= 0.070)].iloc[0]
    except:
        crossover_signal = None
    if crossover_signal is not None:
        print("raju")
#         print(crossover_signal.item)
# df.loc[(df.macd_diff <= -0.070)] # Signal
# df[df.signal.str.endswith("SELL_CROSSOVER")] # Crossover
last_crossover.date - timedelta(minutes=5)

raju


Timestamp('2020-02-05 14:20:00')

In [7]:
# Shochastic Crossover
today_date = datetime.today().date()
df = get_stochastic_data(sorted_stock.symbol)
df.loc[(df["signal"] != df["signal"].shift()) & (df["signal"] == "BUY"), "signal"] = "BUY_CROSSOVER"
df.loc[(df["signal"] != df["signal"].shift()) & (df["signal"] == "SELL"), "signal"] = "SELL_CROSSOVER"
df = df.loc[df["date"] > str(today_date)]
try:
    if sorted_stock.entry_type == "SELL":
        last_crossover = df[df.signal.str.endswith("SELL_CROSSOVER")].iloc[-6]
    elif sorted_stock.entry_type == "BUY":
        last_crossover = df[df.signal.str.endswith("BUY_CROSSOVER")].iloc[-1]
except:
    last_crossover = None
if last_crossover is not None:
    df_after_last_crossover = df.loc[df["date"] > last_crossover.date]
    try:
        if last_crossover.signal == "SELL_CROSSOVER":
            crossover_signal = df_after_last_crossover.loc[(df.stoch_diff <= -22.70)].iloc[0]
        elif last_crossover.signal == "BUY_CROSSOVER":
            crossover_signal = df_after_last_crossover.loc[(df.stoch_diff >= 0.070)].iloc[0]
    except:
        crossover_signal = None
    if crossover_signal is not None and last_crossover is not None:
        print("RAJU")
# df.loc[(df.stoch_diff <= -22.70)] # Signal
df.head(50)

RAJU


,candle_type,open_price,high_price,low_price,close_price,volume,total_buy_quantity,total_sell_quantity,date,stoch,stoch_signal,stoch_diff,percentage,signal
75,M5,167.35,168.95,166.65,168.75,2102532,None,None,2020-02-05 09:15:00,94.736842,69.078947,25.657895,24.307479,BUY
76,M5,168.80,170.20,168.75,170.00,2136125,None,None,2020-02-05 09:20:00,96.039604,83.592149,12.447455,11.954487,BUY
77,M5,169.90,171.35,169.90,171.10,2367161,None,None,2020-02-05 09:25:00,95.967742,95.581396,0.386346,0.370767,BUY
78,M5,171.05,171.20,170.10,170.40,1499273,None,None,2020-02-05 09:30:00,84.677419,92.228255,-7.550836,-6.393853,SELL_CROSSOVER
79,M5,170.50,172.35,170.30,172.10,2412016,None,None,2020-02-05 09:35:00,96.527778,92.390980,4.136798,3.993159,BUY_CROSSOVER
80,M5,171.80,172.55,171.80,171.80,1589978,None,None,2020-02-05 09:40:00,89.864865,90.356687,-0.491822,-0.441976,SELL_CROSSOVER
81,M5,171.75,172.20,171.60,171.95,700786,None,None,2020-02-05 09:45:00,91.891892,92.761512,-0.869620,-0.799110,SELL
82,M5,171.95,173.15,171.25,172.90,1889871,None,None,2020-02-05 09:50:00,96.875000,92.877252,3.997748,3.872818,BUY_CROSSOVER
83,M5,173.00,173.85,172.90,173.40,2040391,None,None,2020-02-05 09:55:00,94.827586,94.531493,0.296094,0.280778,BUY
84,M5,173.30,173.40,172.60,172.90,888759,None,None,2020-02-05 10:00:00,89.080460,93.594349,-4.513889,-4.020993,SELL_CROSSOVER


In [ ]:
# raju = ["adanient", "bhel", "dlf", "gmrinfra", "gail", "infratel", "ioc", "idea", "nhpc", "pnb", "relinfra", "recltd", "spencers", "tatapower", "vakrangee", "yesbank", "ujjivansfb", "granules", "mothersumi", "adanipower", "dishtv", "sail", "nationalum", "jswsteel", "federalbnk", "jindalstel", "idfcfirstb", "rvnl", "equitas"]
# fifth = date(2020,1,25)
# for r in raju:
#     symbol = Symbol.objects.get(symbol=r)
#     SortedStocksList.objects.create(symbol=symbol, entry_type="BUY")
# # Candle.objects.filter(date__date=fifth, symbol__symbol__in=raju)
# for candle in Candle.objects.filter(date__date=fifth, symbol__symbol__in=raju):
#     candle.date = candle.date + timedelta(1)
#     candle.save()

In [ ]:
# import requests
# from bs4 import BeautifulSoup
# # url = "https://www1.nseindia.com/live_market/dynaContent/live_analysis/pre_open/nifty.json"
# url = "https://www1.nseindia.com/live_market/dynaContent/live_analysis/pre_open/preOpenOrderBook.jsp?param=IOCEQN&symbol=IOC"
# payload = {}
# headers= {"Host": "www1.nseindia.com", 
#             "Cache-Control": "max-age=0",
#             "Upgrade-Insecure-Requests": "1", "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36",
#              "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3",
#              }
# # print(response.text.encode('utf8'))
# # url_request.text.encode("utf-8")
# response = requests.get(url, headers=headers)
# soup = BeautifulSoup(response.content)
# soup

In [ ]:
# {'symbol': 'VISESHINFO',
#   'series': 'BE',
#   'xDt': '-',
#   'caAct': '-',
#   'iep': '0.10',
#   'chn': '0.05',
#   'perChn': '100.00',
#   'pCls': '0.05',
#   'trdQnty': '1,01,701',
#   'iVal': '0.10',
#   'mktCap': '9.62',
#   'yHigh': '0.10',
#   'yLow': '0.05',
#   'sumVal': '7,930.97',
#   'sumQnty': '52,80,743',
#   'finQnty': '1,01,701',
#   'sumfinQnty': '56,50,345'}

In [29]:
from django.conf import settings
def import_daily_losers_gainers():
    urls = {
            "BUY": ["https://www1.nseindia.com/live_market/dynaContent/live_analysis/gainers/niftyGainers1.json",
                    "https://www1.nseindia.com/live_market/dynaContent/live_analysis/gainers/jrNiftyGainers1.json"],
            "SELL": ["https://www1.nseindia.com/live_market/dynaContent/live_analysis/losers/niftyLosers1.json",
                    "https://www1.nseindia.com/live_market/dynaContent/live_analysis/losers/jrNiftyLosers1.json"]
        }
    
    nifty_movement = Symbol.objects.get(symbol="nifty_50").get_nifty_movement()
    
    def response_filter(obj):
        open_price = obj.get("openPrice")
        change = float(obj.get("netPrice"))
        if "," in open_price:
            open_price = open_price.replace(",","")
        open_price = float(open_price)
        if open_price >= 100 and open_price <= 300 and change >= 1.2:
            return obj
        
    
    if nifty_movement in ("BUY", "SELL"):
        import_urls = urls.get(nifty_movement)
        if import_urls:
            for url in import_urls:
                response = requests.get(url, headers=settings.NSE_HEADERS)
                if response.status_code == 200:
                    responses = filter(response_filter, response.json().get("data"))
                    for symbol in responses:
                        try:
                            stock = Symbol.objects.get(symbol=symbol.get("symbol").lower())
                        except:
                            stock = None
                        if stock:
                            SortedStocksList.objects.get_or_create(symbol=stock, entry_type=nifty_movement, created_at__date=datetime.now().date())
                    return f"Data imported successfully! from {url}"
                return slack_message_sender.delay(channel="#random", text=f"Incorrect Url: {url}")
            return "All Urls Data Imported Succefully"

tatamotors
jswsteel
vedl
hindalco
infratel
pfc
gicre
nmdc
l&tfh
cadilahc
